In [1]:
print("ello")

ello


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random
import matplotlib.pyplot as plt

# Problem Instance
random.seed(1234)
M = 1000  # Priority weighting for vehicle costs
locations = 10
task_count = 50

# Generate tasks with random demands and locations
tasks = [
    {'weight': random.randint(10, 30),
     'volume': random.randint(10, 30),
     'loc': random.randint(0, locations - 1)}
    for _ in range(task_count)
]

# Vehicle Fleet Parameters
num_large = 50
num_small = 50

# Vehicle specifications
large_vehicle_spec = {'type': 'large', 'weight_cap': 200, 'volume_cap': 300, 'cost': 350}
small_vehicle_spec = {'type': 'small', 'weight_cap': 100, 'volume_cap': 150, 'cost': 200}

# Generate vehicle fleet
vehicles = [large_vehicle_spec.copy() for _ in range(num_large)] + \
           [small_vehicle_spec.copy() for _ in range(num_small)]

# Results storage
cl_values = list(range(9, 0, -1))
objective_values = []

# Gurobi Silent Environment
with gp.Env(empty=True) as env:
    env.setParam('OutputFlag', 1)      # Turn OFF to keep silent
    env.start()

    for C_L in cl_values:
        model = gp.Model('VRP_Gurobi', env=env)

        # Decision Variables
        t = model.addVars(len(tasks), len(vehicles), vtype=GRB.BINARY, name="task_assign")
        o = model.addVars(len(vehicles), vtype=GRB.BINARY, name="vehicle_used")
        d = model.addVars(locations, len(vehicles), vtype=GRB.BINARY, name="location_visit")

        # Objective Function
        vehicle_cost = gp.quicksum(M * vehicles[r]['cost'] * o[r] for r in range(len(vehicles)))
        location_cost = gp.quicksum(d[l, r] for l in range(locations) for r in range(len(vehicles)))
        model.setObjective(vehicle_cost + location_cost, GRB.MINIMIZE)

        # Constraints
        for i in range(len(tasks)):
            model.addConstr(gp.quicksum(t[i, r] for r in range(len(vehicles))) == 1)

        for r in range(len(vehicles)):
            model.addConstr(gp.quicksum(tasks[i]['weight'] * t[i, r] for i in range(len(tasks))) <= vehicles[r]['weight_cap'])
            model.addConstr(gp.quicksum(tasks[i]['volume'] * t[i, r] for i in range(len(tasks))) <= vehicles[r]['volume_cap'])
            model.addConstr(gp.quicksum(d[l, r] for l in range(locations)) <= C_L * o[r])
            model.addConstr(gp.quicksum(t[i, r] for i in range(len(tasks))) <= len(tasks) * o[r])

        for l in range(locations):
            for r in range(len(vehicles)):
                model.addConstr(gp.quicksum(t[i, r] for i in range(len(tasks)) if tasks[i]['loc'] == l) <= len(tasks) * d[l, r])

        # Solve for 100% optimality
        model.Params.MIPGap = 0.0

        # Solve
        model.optimize()

        # Collect objective value
        if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL:
            objective_values.append(model.ObjVal)
        else:
            objective_values.append(None)  # Use None for infeasible or other status

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(cl_values, objective_values, marker='o')
plt.xlabel("C_L (Max Locations per Vehicle)")
plt.ylabel("Objective Value")
plt.title("Objective Value vs. C_L")
plt.grid(True)
plt.gca().invert_xaxis()  # Optional: Show decreasing C_L left to right
plt.tight_layout()
plt.show()
print(objective_values)

Set parameter Username
Set parameter LicenseID to value 2616800
Academic license - for non-commercial use only - expires 2026-01-31
Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
MIPGap  0

Optimize a model with 1450 rows, 6100 columns and 27200 nonzeros
Model fingerprint: 0x22a4cd61
Variable types: 0 continuous, 6100 integer (6100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 1.185005e+07
Presolve removed 200 rows and 200 columns
Presolve time: 0.05s
Presolved: 1250 rows, 5900 columns, 26800 nonzeros
Variable types: 0 continuous, 5900 integer (5900 binary)

Root relaxation: object